**Description**: Show how to construct basic IP formulations for matching in causal inference

**Author**: Juan Pablo Vielma

**License**: <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

# IP formulations for matching in causal inference with JuMP

We first consider the matching problem of the form
$$\begin{alignedat}{4}
 &\min & \quad  \sum_{t=1}^{T} \sum_{c=1}^{C} m_{t,c}  \\
 \notag&s.t.\\
 &      &  \sum_{t=1}^{T} m_{t,c} &\leq 1,  &\quad& \forall c\in \{1,\ldots,C\} \\
  &      &   \sum_{c=1}^{C} m_{t,c} &\leq 1,  &\quad& \forall d\in \{1,\ldots,T\} \\
  && m_{t,c}&=0, &\quad& \forall t,c \quad \mathbf{x}^t\neq \mathbf{x}^c\\
  &      &  m_{t,c} &\geq 0,  &\quad& \forall t\in \{1,\ldots,T\},\quad c\in \{1,\ldots,C\} \\
 \end{alignedat}$$
 and then the matching problem of the form 
 $$\begin{alignedat}{4}
 &\min & \quad  \sum_{t=1}^{T} \sum_{c=1}^{C} m_{t,c}  \\
 \notag&s.t.\\
 &      &  \sum_{t=1}^{T} m_{t,c} &\leq 1,  &\quad& \forall c\in \{1,\ldots,C\} \\
  &      &   \sum_{c=1}^{C} m_{t,c} &\leq 1,  &\quad& \forall d\in \{1,\ldots,T\} \\
  && \sum_{t=1}^{T}\sum_{c=1}^{C} \mathbf{x}^t_p m_{t,c}&== \sum_{t=1}^{T}\sum_{c=1}^{C} \mathbf{x}^c_p m_{t,c}&\quad& \forall p\in \{1,\ldots,P\}\\
  &      &  m_{t,c} &\in \{0,1\},  &\quad& \forall t\in \{1,\ldots,T\},\quad c\in \{1,\ldots,C\} \\
 \end{alignedat}$$

In [1]:
# Auxiliary function to generate covariate data
function generateCovariates(T,C,P,K)
    xt = rand(1:K,T,P)
    xc = rand(1:K,C,P)
    xt, xc
end

generateCovariates (generic function with 1 method)

In [2]:
xt, xc = generateCovariates(3,5,2,2)

(
3x2 Array{Int64,2}:
 1  1
 1  1
 1  1,

5x2 Array{Int64,2}:
 2  2
 2  2
 2  1
 1  2
 2  2)

In [3]:
using JuMP, GLPKMathProgInterface

In [4]:
# Create JuMP model, using GLPK as the solver
model = Model(solver=GLPKSolverLP())

# Data size
T = size(xt,1)
C = size(xc,1)
P = size(xt,2)

# Define variables
@variable(model, m[1:T,1:C] >= 0)

# Add constraints
@constraints(model, begin    
    supply_constraints[c=1:C], sum{m[t,c],  t=1:T} <= 1
    demand_constraints[t=1:T], sum{m[t,c],  c=1:C} <= 1
    zero_constraints[t=1:T,c=1:C; xt[t,:] != xc[c,:]], m[t,c] == 0
end)

# Define objective
@objective(model, Max, sum{ m[t,c], t=1:T,c=1:C})

# Display Model
println(model)

# Solve Model
println("Solving...")
status = solve(model)

# Display results
println("Solver status: ", status)
println("Cost: ", getobjectivevalue(model))
println("Solution: \n",getvalue(m))

Max m[1,1] + m[1,2] + m[1,3] + m[1,4] + m[1,5] + m[2,1] + m[2,2] + m[2,3] + m[2,4] + m[2,5] + m[3,1] + m[3,2] + m[3,3] + m[3,4] + m[3,5]
Subject to
 m[1,1] + m[2,1] + m[3,1] ≤ 1
 m[1,2] + m[2,2] + m[3,2] ≤ 1
 m[1,3] + m[2,3] + m[3,3] ≤ 1
 m[1,4] + m[2,4] + m[3,4] ≤ 1
 m[1,5] + m[2,5] + m[3,5] ≤ 1
 m[1,1] + m[1,2] + m[1,3] + m[1,4] + m[1,5] ≤ 1
 m[2,1] + m[2,2] + m[2,3] + m[2,4] + m[2,5] ≤ 1
 m[3,1] + m[3,2] + m[3,3] + m[3,4] + m[3,5] ≤ 1
 m[1,1] = 0
 m[1,2] = 0
 m[1,3] = 0
 m[1,4] = 0
 m[1,5] = 0
 m[2,1] = 0
 m[2,2] = 0
 m[2,3] = 0
 m[2,4] = 0
 m[2,5] = 0
 m[3,1] = 0
 m[3,2] = 0
 m[3,3] = 0
 m[3,4] = 0
 m[3,5] = 0
 m[i,j] ≥ 0 ∀ i ∈ {1,2,3}, j ∈ {1,2,…,4,5}

Solving...
Solver status: Optimal
Cost: 0.0
Solution: 
[-0.0 -0.0 -0.0 -0.0 -0.0
 -0.0 -0.0 -0.0 -0.0 -0.0
 -0.0 -0.0 -0.0 -0.0 -0.0]


In [5]:
# Create JuMP model, using GLPK as the solver
model = Model(solver=GLPKSolverLP())

# Data size
T = size(xt,1)
C = size(xc,1)
P = size(xt,2)

# Define variables
@variable(model, m[t=1:T,c=1:C; xt[t,:] == xc[c,:]] >= 0)

# Add constraints
@constraints(model, begin    
    supply_constraints[c=1:C], sum{m[t,c],  t=1:T; xt[t,:] == xc[c,:]} <= 1
    demand_constraints[t=1:T], sum{m[t,c],  c=1:C; xt[t,:] == xc[c,:]} <= 1
end)

# Define objective
@objective(model, Max, sum{ m[t,c], t=1:T,c=1:C; xt[t,:] == xc[c,:]})

# Display Model
println(model)

# Solve Model
println("Solving...")
status = solve(model)

# Display results
println("Solver status: ", status)
println("Cost: ", getobjectivevalue(model))
println("Solution: \n",getvalue(m))

LoadError: LoadError: ArgumentError: collection must be non-empty
while loading In[5], in expression starting on line 22

In [6]:
xt, xc = generateCovariates(10,20,2,5)

(
10x2 Array{Int64,2}:
 1  1
 1  2
 2  2
 4  1
 1  5
 5  4
 5  4
 4  1
 5  3
 1  4,

20x2 Array{Int64,2}:
 1  4
 2  3
 4  3
 4  3
 5  5
 1  3
 5  5
 1  5
 4  4
 1  4
 4  2
 4  2
 4  5
 2  1
 3  5
 1  4
 3  4
 3  2
 4  3
 1  4)

In [7]:
# Create JuMP model, using GLPK as the solver
model = Model(solver=GLPKSolverLP())

T = size(xt,1)
C = size(xc,1)
P = size(xt,2)

# Define variables
@variable(model, m[1:T,1:C] >= 0)

# Add constraints
@constraints(model, begin    
    supply_constraints[c=1:C], sum{m[t,c],  t=1:T} <= 1
    demand_constraints[t=1:T], sum{m[t,c],  c=1:C} <= 1
    average_constraints[p=1:P],    sum{xt[t,p]*m[t,c], t=1:T, c=1:C} == sum{xc[c,p]*m[t,c], t=1:T, c=1:C} 
end)

# Define objective
@objective(model, Max, sum{ m[t,c], t=1:T,c=1:C})

# Solve Model
println("Solving...")
status = solve(model)

# Display results
println("Solver status: ", status)
println("Cost: ", getobjectivevalue(model))
println("Solution: \n",getvalue(m))

Solving...
Solver status: Optimal
Cost: 10.0
Solution: 
[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -1.1102230246251575e-16 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 -0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.6249999999999999 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0 0.3750000000000001 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.25000000000000006 0.7499999999999999 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 -0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.7499999999999999 0.0 0.0 0.0 0.0 0.0 0.0 0.25000000000000006 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0
 0.0 0.0 0.0 0.25000000000000006 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.7499999999999999 0.0
 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 -0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.

In [8]:
model = Model(solver=GLPKSolverMIP())

T = size(xt,1)
C = size(xc,1)
P = size(xt,2)

# Define variables
@variable(model, m[1:T,1:C] >= 0, Int)

# Add constraints
@constraints(model, begin    
    supply_constraints[c=1:C], sum{m[t,c],  t=1:T} <= 1
    demand_constraints[t=1:T], sum{m[t,c],  c=1:C} <= 1
    average_constraints[p=1:P],    sum{xt[t,p]*m[t,c], t=1:T, c=1:C} == sum{xc[c,p]*m[t,c], t=1:T, c=1:C} 
end)

# Define objective
@objective(model, Max, sum{ m[t,c], t=1:T,c=1:C})

# Solve Model
println("Solving...")
status = solve(model)

# Display results
println("Solver status: ", status)
println("Cost: ", getobjectivevalue(model))
println("Solution: \n",getvalue(m))

Solving...
Solver status: Optimal
Cost: 10.0
Solution: 
[0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0
 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
